In [57]:
from datetime import datetime, timezone
import requests

parameters = {
    "subreddit": "europe",
    "q":"-moscopole -moscone -muscovite mosco*|kremlin*|russia*|putin*",
    "after": int(datetime(2022, 1, 24, tzinfo=timezone.utc).timestamp()),
    "order": "asc",
    "metadata": True,
    "size": 1000
    
}

endpoint = "https://api.pushshift.io/reddit/search/comment/"
# query = "&".join([f"{key}={value}" for key, value in parameters.items()])
# url = endpoint + "?" + query

response = requests.get(endpoint, params=parameters)
print(response.status_code)
response = response.json()

200


In [60]:
import pandas as pd


comments = pd.DataFrame(comments)
metadata_ = response["metadata"]

comments["created_utc_str"] = comments.created_utc.apply(timestamp_to_ISO8601)
rel_columns = ["created_utc", "created_utc_str", "author_flair_css_class", "author_flair_text", "body"]
with pd.option_context("display.max_colwidth", None):
    display(comments[rel_columns])

,created_utc,created_utc_str,author_flair_css_class,author_flair_text,body
0,1642982447,2022-01-24T00:00:47+00:00,NORW,Norway,"I almost have to wonder if buying the F-35 comes with some unstated guaranteed US protection. I mean, political capital aside, I can only assume the US would do everything to prevent the F-35 falling into Russia or China’s hands, meaning they wouldnt let a country who posses them fall"
1,1642982497,2022-01-24T00:01:37+00:00,None,None,"NATO is a liability for countries like Ukraine given that NATO's main ""enemy"" was and is Russia. Might as well just paint a giant target on their back. \nIf war ever gets fought, it will be on the land of those countries which serves as a buffer zone. \n\nAssuming no major escalation, it won't be American or Russian cities that will get wrecked. But those countries stuck in the middle. Syria 2.0 for them. \n\nIf the people there don't push their braindead politicians to figure out a way to stay neutral...well...joke's will be on them if war ever happens. By then they will wish they've done more. Oh...and those braindead politicians? Will be the first to flee for safer grounds. Many of them probably already got bank accounts and houses parked on nice little exotic islands."
2,1642982847,2022-01-24T00:07:27+00:00,None,None,"No. I agree with providing aid and maybe some reinforcement. However, there is a 99% chance they take Ukraine, and that means Russia will have our weapons."
3,1642983000,2022-01-24T00:10:00+00:00,None,None,"Actually its because of the broken english, opinion that everything europe does is guided by the us, and support of the russian imperialism.\n\nif they arent a russian bot, then they must just be pathetic"
4,1642983065,2022-01-24T00:11:05+00:00,EURO,Limburg,Then why does Russia keep brushing aside the security concerns of Ukraine that make it consider to join NATO?
...,...,...,...,...,...
993,1643040667,2022-01-24T16:11:07+00:00,None,None,"Ah, it’s been a while since we've had a Russo-Turkish war. Maybe we’ll get to hear Putin say Tsargrad."
994,1643040682,2022-01-24T16:11:22+00:00,None,None,"Except for passing Nurremberg laws, politically and militarily Putin has done everything pre-WW2 Hitler did"
995,1643040767,2022-01-24T16:12:47+00:00,POLA,Polska,It wasn't Russian/Muscovy territory at that time. Those were wild fields under control of Tatars.
996,1643040788,2022-01-24T16:13:08+00:00,None,None,"""You can't ever compare any dictator to Hitler""\n\nYes, we can. We can even compare 1920's Hitler to Putin, or 1933's Hitler to Trump, or 1945's Hitler to you. Some comparisons make a lot of sense, others don't. \n\nBut comparing 1938 Hitler to 2022 Putin? That's pretty fair."


In [61]:
import pprint

pp = pprint.PrettyPrinter()
pp.pprint(metadata)

{'api_launch_time': 1673017478.254743,
 'api_request_end': 1674047192.4329264,
 'api_request_start': 1674047191.615821,
 'api_total_time': 0.8171055316925049,
 'es': {'_shards': {'failed': 0,
                    'skipped': 820,
                    'successful': 824,
                    'total': 824},
        'hits': {'max_score': None, 'total': {'relation': 'eq', 'value': 2039}},
        'timed_out': False,
        'took': 712},
 'es_query': {'aggs': {},
              'query': {'bool': {'must': [{'bool': {'must': [{'simple_query_string': {'default_operator': 'and',
                                                                                      'fields': ['body'],
                                                                                      'query': '-moscopole '
                                                                                               '-moscone '
                                                                                               'mosco*|kre

In [35]:
comments.dtypes

all_awardings                       object
archived                              bool
associated_award                    object
author                              object
author_created_utc                 float64
author_flair_background_color       object
author_flair_css_class              object
author_flair_richtext               object
author_flair_template_id            object
author_flair_text                   object
author_flair_text_color             object
author_flair_type                   object
author_fullname                     object
author_patreon_flair                object
author_premium                      object
body                                object
can_gild                              bool
collapsed                             bool
collapsed_because_crowd_control     object
collapsed_reason                    object
collapsed_reason_code               object
comment_type                        object
controversiality                     int64
created_utc

In [36]:
from datetime import datetime
comments.created_utc.apply(datetime.fromtimestamp)

0     2022-01-24 02:00:47
1     2022-01-24 02:01:37
2     2022-01-24 02:07:27
3     2022-01-24 02:10:00
4     2022-01-24 02:11:05
              ...        
993   2022-01-24 18:11:07
994   2022-01-24 18:11:22
995   2022-01-24 18:12:47
996   2022-01-24 18:13:08
997   2022-01-24 18:14:19
Name: created_utc, Length: 998, dtype: datetime64[ns]

In [37]:
datetime.fromtimestamp(1674044681)

datetime.datetime(2023, 1, 18, 14, 24, 41)

In [69]:
import time
from datetime import timedelta
from functools import partial, wraps


def timeit(f):
    @wraps(f)
    def wrap(*args, **kwargs):
        t0 = time.time()
        result = f(*args, **kwargs)
        t1 = time.time()
        print(("Function {f_name} args:[{args}, {kwargs}] took: {time:.2f} secs."
                   .format(f_name=f.__name__, args=args, kwargs=kwargs, time=t1-t0)))
        return result
    return wrap


def timestamp_to_ISO8601(timestamp):
    dt_obj = datetime.fromtimestamp(timestamp)
    return dt_obj.astimezone(tz=timezone.utc).isoformat()


def format_query_params(after, before=None, include_metadata=False,
                        query_str="-moscopole -moscone mosco*|kremlin*|russia*|putin*",
                        default_params=dict(subreddit="europe", size=500)):
                        
    params = {**default_params, "q": query_str, "metadata": include_metadata, 
              "after": after, "order": "asc"}
    if before is not None:
        params["before"] = before
    return params


def send_request(parameters, timeout=10, endpoint="https://api.pushshift.io/reddit/search/comment/"):
    response = requests.get(endpoint, params=parameters)
    
    if response.status_code == 200:
        response = response.json()
        metadata = response["metadata"] if "metadata" in response.keys() else None
        data = response["data"]
        return (metadata, data)
    
    elif response.status_code == 429:
        print(f"Rate limit reached, sleeping for {timeout} secs.")
        time.sleep(timeout)
        return send_request(parameters)
    
    elif response.status_code >= 500:
        print(f"Server error (HTTP {response.status_code}), sleeping for {timeout} secs.")
        time.sleep(timeout)
        return send_request(parameters)
    
    else:
        raise NotImplementedError("HTTP status code {}".format(response.status_code))
    
@timeit   
def get_daily_comments(date_str):
    after_datetime = datetime.fromisoformat(date_str).replace(tzinfo=timezone.utc)
    before_datetime = after_datetime + timedelta(days=1)
    params = format_query_params(after=int(after_datetime.timestamp()),
                                 before=int(before_datetime.timestamp()),
                                 include_metadata=True)
    
    format_subseq_query_params = partial(format_query_params, 
                                         before=int(before_datetime.timestamp()))
    
    metadata, comments = send_request(params)
    
    iteration = 0
    print(f"#{iteration+1}: {len(comments)} ({len(comments)}/{metadata['es']['hits']['total']['value']})")
    print(timestamp_to_ISO8601(comments[-1]["created_utc"]))
    while True:
        params = format_subseq_query_params(after=int(comments[-1]["created_utc"]) + 1)
        _, data = send_request(params)
        
        if bool(data):
            comments += list(data)
            iteration += 1
            print(f"#{iteration+1}: {len(data)} ({len(comments)}/{metadata['es']['hits']['total']['value']})")
            print(timestamp_to_ISO8601(data[-1]["created_utc"]))
        else:
            break
    
    return metadata, comments

In [72]:
metadata, comments = get_daily_comments("2023-01-15")

#1: 499 (499/1404)
2023-01-15T12:07:43+00:00
#2: 499 (998/1404)
2023-01-15T17:10:36+00:00
#3: 404 (1402/1404)
2023-01-15T23:59:34+00:00
Function get_daily_comments args:[('2023-01-15',), {}] took: 6.77 secs.


In [73]:
comments = pd.DataFrame(comments)

comments["created_utc_str"] = comments.created_utc.apply(timestamp_to_ISO8601)
rel_columns = ["created_utc", "created_utc_str", "author_flair_css_class", "author_flair_text", "body"]
with pd.option_context("display.max_colwidth", None):
    display(comments[rel_columns])

,created_utc,created_utc_str,author_flair_css_class,author_flair_text,body
0,1673740805,2023-01-15T00:00:05+00:00,FINL,Finland,"Finland alone has 900 000 trained men &amp; women in the reserves and the largest artillery in western europe. Together we have over 300 mbts and almost 300 fighter aircraft. With the swedish navy, a chokepoint can be made between helsinki and tallin which prevents russian ships from sailing between st petersburg and kaliningrad. And with the baltic sea closed off of russians the baltic states can still be supplied, armed and supported from the sea even if the russians manage to close the suvalki gap, not to even mention the amount of pressure that is taken off of baltic state's shoulders when russia has to monitor 1300km of new border with nato-finland. You are seriously misinformed if you think humanism and money are the only things sweden and finland bring to the table"
1,1673740850,2023-01-15T00:00:50+00:00,None,None,"We need to kick Turkey from NATO, they’re just as bad as Russia by this point especially if they invade Syria again"
2,1673740896,2023-01-15T00:01:36+00:00,GERM,𝔊𝔲𝔱𝔢𝔫 𝔗𝔞𝔤!,"Fucking tankie, move to Russia"
3,1673741073,2023-01-15T00:04:33+00:00,UA-32,Kyiv region (Ukraine),&gt;that Ukraine and Russia formed one nation not so long ago\n\nWhich one is that?
4,1673741121,2023-01-15T00:05:21+00:00,TURK,Turkey,"To be honest, I don't know about this in a completely neutral way. I only know that two states became enemies in history (I learned in history class) How and why they became enemies According to the scale they teach in Turkey, our hostility ""started with helping the Russians"", …"
...,...,...,...,...,...
1397,1673826850,2023-01-15T23:54:10+00:00,None,None,&gt; And have you missed the repeated friendly warnings the Russian federation has made to African nations over their wavering support?\n\nSuch as ?
1398,1673826869,2023-01-15T23:54:29+00:00,UA-30,Kyiv (Ukraine),"https://twitter.com/SiegeOfTerra/status/1614583199136350209\n\n&gt;In 2023 Games Workshop plans to release two AAA-games in Warhammer40k setting:\n1. Space Marine 2, made in Russia by @SaberGames, published by genocide supporters @Focus_entmt\n \n2. Rogue Trader, made in Russia by kremlin-funded Owlcat\n\nPLEASE SPREAD THE WORD AND BOYCOTT THESE GAMES\n\nthis post made me permabanned from /r/worldnews, let's see if the mods of /r/europe are more sane"
1399,1673827058,2023-01-15T23:57:38+00:00,None,None,That’s so Russian
1400,1673827107,2023-01-15T23:58:27+00:00,ENGL,Glorious Revolution II: Parliament Edition Please,"&gt; cynicism \n\nDo you expect me to have optimism? Tell me. Name a war where civilians have not been affected. It is not excuse, it is not cynicism it is fact. War does not care whether you are innocent or complicit. Meek men do not make good soldiers. Hard men, violent men, imperfect men do. Under what circumstances do you expect them to abide by the 'rules' of war when they see the very worst actions mankind has to offer? I'd be pretty fucked off if I was in their shoes too.\n\nAgain. The only way to avoid breaking the rules of war is to not start a war. This is a fact. The instigator creates the condition and the opportunity for civilian casualties. They and they alone are responsible.\n\nIn summary. Shit happens in war and I'd rather it happen to Russians in this instance. Fuck em."


In [56]:
pp.pprint(metadata)

{'api_launch_time': 1673017478.254743,
 'api_request_end': 1674047192.4329264,
 'api_request_start': 1674047191.615821,
 'api_total_time': 0.8171055316925049,
 'es': {'_shards': {'failed': 0,
                    'skipped': 820,
                    'successful': 824,
                    'total': 824},
        'hits': {'max_score': None, 'total': {'relation': 'eq', 'value': 2039}},
        'timed_out': False,
        'took': 712},
 'es_query': {'aggs': {},
              'query': {'bool': {'must': [{'bool': {'must': [{'simple_query_string': {'default_operator': 'and',
                                                                                      'fields': ['body'],
                                                                                      'query': '-moscopole '
                                                                                               '-moscone '
                                                                                               'mosco*|kre